In [ ]:
import re
import time
import os
import pandas as pd
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- Configurações iniciais ---

dotenv_path = "../data/secure/.env"
load_dotenv(dotenv_path)
EMAIL = os.getenv("LOGIN")
SENHA = os.getenv("SENHA_ISSE")

chromedriver_path = r"C:\Users\User\Desktop\Repositorios\Automações\src\others\chromedriver.exe"

options = Options()
options.add_argument("--start-maximized")
service = Service(executable_path=chromedriver_path)
driver = webdriver.Chrome(service=service, options=options)

# --- Login ---

driver.get("https://auth.maringa.ecity.com.br/v2/sign_in?return_to=https%3A%2F%2Fauth.maringa.ecity.com.br%2Fv2%2Foauth2%2Fauthorize%3Fresponse_type%3Dcode%26client_id%3Dba1c97e995f70%26redirect_uri%3Dhttps%253A%252F%252Fmaringa.fintel.com.br%252FAccount%252FSenhaWeb%26scope%3D%26state%3DNfs")

WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "login"))).send_keys(EMAIL)
driver.find_element(By.ID, "password").send_keys(SENHA)
driver.find_element(By.ID, "password").send_keys(Keys.RETURN)
time.sleep(5)

# --- Lê os dados da planilha ---
df = pd.read_csv("../data/output/clientes_com_cpf.csv")

# DataFrames para guardar resultados
df_sem_cpf = pd.DataFrame(columns=df.columns)
df_notas_lancadas = pd.DataFrame(columns=df.columns)

for index, row in df.iterrows():
    time.sleep(3)
    driver.get("https://maringa.fintel.com.br/ConsultaNotasFiscaisEmitidas")
    time.sleep(3)
    cpf = row['CPF']
    valor = float(row['VALOR'])
    imposto = round(valor * 0.1772, 2)

    if cpf == "000.000.000-00":
        print(f"CPF inválido encontrado, pulando: {cpf} (linha {index})")
        df_sem_cpf = pd.concat([df_sem_cpf, pd.DataFrame([row])], ignore_index=True)
        continue

    print(f"Processando CPF: {cpf} | Valor: {valor} | Imposto: R${imposto:.2f}")
    time.sleep(3)
    btn_emitir = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//a[@href='/Emissor/Nfse' and contains(@class, 'btn-primary')]"))
    )
    btn_emitir.click()
    time.sleep(3)
    # time.sleep(1)  # para visualizar passo a passo

    input_cpf = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.ID, "Tomador_CnpjCpf"))
    )
    input_cpf.clear()
    input_cpf.send_keys(cpf)

    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "btnBuscarTomador"))
    ).click()
    time.sleep(1.5)  # necessário aguardar o preenchimento

    cep = driver.find_element(By.ID, "tomador_Cep").get_attribute("value")
    logradouro = driver.find_element(By.ID, "tomador_Logradouro").get_attribute("value")
    numero = driver.find_element(By.ID, "tomador_Numero").get_attribute("value")

    if not cep or not logradouro or not numero:
        if not cep:
            driver.find_element(By.ID, "tomador_Cep").clear()
            driver.find_element(By.ID, "tomador_Cep").send_keys("87013-060")
            WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(@onclick, 'Emissor_BuscarCep')]"))
            ).click()
            time.sleep(1)

        if not numero:
            driver.find_element(By.ID, "tomador_Numero").clear()
            driver.find_element(By.ID, "tomador_Numero").send_keys("123")
            time.sleep(0.5)
    servico_select = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "IdServico"))
    )
    for option in servico_select.find_elements(By.TAG_NAME, 'option'):
        if "050802" in option.text:
            option.click()
            break
    descricao = f"Prestação de serviços. Informação aproximada de tributação R${imposto:.2f} (17.72%) conforme lei 12.741/2012 fonte IBPT."
    descricao_input = driver.find_element(By.ID, "Descricao")
    descricao_input.clear()
    descricao_input.send_keys(descricao)
    valor_input = driver.find_element(By.ID, "VlrUnitario")
    valor_input.clear()
    valor_input.send_keys(f"{valor:.2f}".replace('.', ','))

    # Em vez de clicar em confirmar, clicamos em Voltar para teste
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "buttonConfirmar"))
    ).click()

    print(f"Feito teste para CPF {cpf} - clicou em confirmar.")

    time.sleep(2)
    # Guarda os dados processados com sucesso
    df_notas_lancadas = pd.concat([df_notas_lancadas, pd.DataFrame([row])], ignore_index=True)

    driver.get("https://maringa.fintel.com.br/ConsultaNotasFiscaisEmitidas")

    time.sleep(3)  # comente esse sleep depois para acelerar

# Salva arquivos de saída
df_sem_cpf.to_csv("../data/output/sem_cpf.csv", index=False)
df_notas_lancadas.to_csv("../data/output/notas_lancadas.csv", index=False)

print("✅ Teste finalizado. Arquivos 'sem_cpf.csv' e 'notas_lancadas.csv' gerados.")
